# Nibiru Spots (Swaps) Queries and Transactions
Examples of executing asset swaps.

### 1. Install Nibiru package

In [ ]:
!pip install nibiru

### 2. Create chain client and run queries
Using testnet-3 (aka itn-3) endpoint run queries.

In [ ]:
import json
from nibiru import Network, ChainClient, Msg, Coin, TxConfig
from nibiru.exceptions import QueryError

client = ChainClient(network=Network.testnet(3))

In [ ]:
# Shortcut to print responses
def print_json(obj):
    print(json.dumps(obj, indent=2))

In [ ]:
# Query spot params (fees and whitelisted assets)
spot_params = client.query.spot.params()
print_json(spot_params)

In [ ]:
# Query spot pools
pools = client.query.spot.pools()
print_json(pools)

### 3. Authenticate to run transactions
Assuming you have your wallet mnemonic key (or private key) and some test tokens from the faucet.
See: https://app.nibiru.fi/faucet

In [ ]:
mnemonic = "put your mnemonic here..."

# Let's set high gas multiplier to avoid out of gas errors. Could be customized per tx.
tx_config = TxConfig(gas_multiplier=5)
client = ChainClient(Network.testnet(3), tx_config=tx_config)
client.authenticate(mnemonic=mnemonic)

# Balances check
balances = client.query.get_bank_balances(client.address)
print_json(balances["balances"])

### 4. Run Spot transactions
Swap tokens:

In [ ]:
pool_id = 1  # select the pool from the query above
token_in = "unusd"  # check that token is in the pool
token_out = "uusdt" # check that token is in the pool

output = client.tx.execute_msgs(
    Msg.spot.swap(
        pool_id=pool_id,
        token_in=Coin(10, token_in),
        token_out_denom=token_out,
    ),
    wait_for_tx_resp=True,  # wait for block and get tx response
)
# Expected tx response code 0
print(f"tx response Code: {output['code']}")
if output["code"] == 0:
    balances = client.query.get_bank_balances(client.address)
    print_json(balances["balances"])

Join spot pool. After joining the pool you will have a pool shares in your balance with asset name like "nibiru/pool/1".

In [ ]:
output = client.tx.execute_msgs(
    Msg.spot.join_pool(
        pool_id=pool_id,
        tokens=[Coin(10, token_in), Coin(5, token_out)]
    ),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    balance = client.query.get_bank_balance(client.address, f"nibiru/pool/{pool_id}")
    print_json(balance["balance"])
    total_shares_amount = balance["balance"]["amount"]

Exit pool. Pool shares will disappear from balances list.

In [ ]:
output = client.tx.execute_msgs(
    Msg.spot.exit_pool(
        pool_id=pool_id,
        pool_shares=Coin(total_shares_amount, f"nibiru/pool/{pool_id}")
    ),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    balances = client.query.get_bank_balances(client.address)
    print_json(balances["balances"])